Hanel et al., iScience 2025

### Fig. 3: PCA - clustering efficiency metrics

In [1]:
import scanpy as sc
import os
import scib
import numpy as np
import pandas as pd

To evaluate how well are different cell types separated in latent representation; scIB package from Theis lab https://www.nature.com/articles/s41592-021-01336-8

Sources: 
- scIB: https://scib.readthedocs.io/en/latest/user_guide.html#embedding-output
- Theis sc best practices: https://www.sc-best-practices.org/cellular_structure/integration.html
- scib-metrics: https://scib-metrics.readthedocs.io/en/latest/notebooks/lung_example.html

In [2]:
ad = sc.read_h5ad("./data/Level1_pbulk_192samples.h5ad")

ad.obs["cell_type"] = ad.obs["Level 1"]

# Filter out Ba/Ma/Eo
#exclude = ["Ba/Ma/Eo"]
#ad = ad[~ad.obs["cell_type"].isin(exclude)].copy()
ad

AnnData object with n_obs × n_vars = 192 × 20898
    obs: 'donor_id', 'Level 1', 'study', 'sex', 'age', 'disease', 'disease_category', 'age_category', 'age_ontology', 'blasts_pct', 'risk', 'psbulk_n_cells', 'psbulk_counts', 'cell_type'
    var: 'gene_ids', 'gene_count', 'ensembl_gene_id', 'entrezgene_id', 'external_gene_name', 'hgnc_symbol', 'description', 'chromosome_name', 'start_position', 'end_position', 'gene_biotype', 'strand', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    layers: 'psbulk_props'

In [3]:
desired_order = [
    "HSC",
    "MPP",
    "MultiLin",
    "MEP",
    "early-Erythroid",
    "late-Erythroid",
    "MKP",
    "CLP",
    "Transitional-B",
    "pre-B",
    "B cell",
    "Plasma Cell",
    "CD4 T-cell",
    "T/NK",
    "GMP",
    "early-Neu",
    "early-Mono",
    "Monocyte",
   "Myeloid intermediate",
    "MDP",
    "pre-DC",
    "cDC",
    "pDC",
    "ASDC"
]

ad.obs['cell_type'] = pd.Categorical(ad.obs['cell_type'], 
                                     categories=desired_order, ordered=True)

In [4]:
directory_path = "/research/work/andreha/privat/Andrea/2025_Sylvain/data/"
all_files = os.listdir(directory_path)

filtered_files = [file for file in all_files if "pca" in file and ".csv" in file]
filtered_files

['pca_CC24.csv', 'pca_hvg24.csv', 'pca_receptors24.csv', 'pca_random24.csv']

In [5]:
samples = [name.replace('pca_', '').replace('.csv', '') for name in filtered_files]
print(samples)

['CC24', 'hvg24', 'receptors24', 'random24']


In [6]:
for filename in filtered_files:
    pca_path = os.path.join(directory_path, filename)
    pca = pd.read_csv(pca_path)
    print(f"\n--- {filename} ---")
    print(pca[['sample', 'x', 'y']].head())



--- pca_CC24.csv ---
     sample         x         y
0  BM1_ASDC -2.166090 -1.302319
1  BM2_ASDC -5.285326  0.593405
2  BM3_ASDC -1.425685 -1.822142
3  BM4_ASDC -3.427387 -0.695913
4  BM5_ASDC -1.269101 -2.316622

--- pca_hvg24.csv ---
     sample         x         y
0  BM1_ASDC  0.833092 -2.754932
1  BM2_ASDC  0.526365 -3.593792
2  BM3_ASDC -0.076201 -3.540178
3  BM4_ASDC -0.055868 -3.385817
4  BM5_ASDC -0.000705 -2.221834

--- pca_receptors24.csv ---
     sample         x         y
0  BM1_ASDC  1.381923  0.274668
1  BM2_ASDC  2.195384  0.301616
2  BM3_ASDC  1.725118  0.202207
3  BM4_ASDC  4.196408  0.291791
4  BM5_ASDC  2.964408 -1.131064

--- pca_random24.csv ---
     sample         x         y
0  BM1_ASDC -0.583672 -6.379669
1  BM2_ASDC -2.223347 -2.826207
2  BM3_ASDC -1.368097 -4.763889
3  BM4_ASDC  0.178734 -2.347719
4  BM5_ASDC -1.196480 -4.519464


In [7]:
directory_path = "/research/work/andreha/privat/Andrea/2025_Sylvain/data/"

all_files = os.listdir(directory_path)

filtered_files = [file for file in all_files if "pca" in file and ".csv" in file]

adata_list = []

for filename in filtered_files:
    try:
        # Load PCA coordinates from CSV file
        pca = pd.read_csv(os.path.join(directory_path, filename))
        pca.set_index('sample', inplace=True)

        ad_copy = ad.copy()

        ad_copy.obsm["pca"] = pca[['x', 'y']].values

        sc.pp.neighbors(ad_copy, use_rep="pca")
        scib.me.cluster_optimal_resolution(ad_copy, cluster_key="cluster", label_key="cell_type")

        adata_list.append(ad_copy)
    except Exception as e:
        print(f"Error processing {filename}: {e}")



/research/users/andreha/miniconda3/envs/traj_inf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cluster for cluster_0.2 with leiden
resolution: 0.2, nmi: 0.546855161999393
Cluster for cluster_0.4 with leiden
resolution: 0.4, nmi: 0.6371833052927749
Cluster for cluster_0.6 with leiden
resolution: 0.6, nmi: 0.6371833052927749
Cluster for cluster_0.8 with leiden
resolution: 0.8, nmi: 0.6775322190700798
Cluster for cluster_1.0 with leiden
resolution: 1.0, nmi: 0.7263612650141438
Cluster for cluster_1.2 with leiden
resolution: 1.2, nmi: 0.7260558805326924
Cluster for cluster_1.4 with leiden
resolution: 1.4, nmi: 0.7188688113869106
Cluster for cluster_1.6 with leiden
resolution: 1.6, nmi: 0.7188688113869106
Cluster for cluster_1.8 with leiden
resolution: 1.8, nmi: 0.7134692808960197
Cluster for cluster_2.0 with leiden
resolution: 2.0, nmi: 0.7134692808960197
optimised clustering against cell_type
optimal cluster resolution: 1.0
optimal score: 0.7263612650141438
Cluster for cluster_0.2 with leiden
resolution: 0.2, nmi: 0.622537113086464


/research/users/andreha/miniconda3/envs/traj_inf/lib/python3.12/site-packages/scib/metrics/clustering.py:96: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  cluster_function(adata, resolution=res, key_added=resolution_key, **kwargs)


Cluster for cluster_0.4 with leiden
resolution: 0.4, nmi: 0.6826437877693478
Cluster for cluster_0.6 with leiden
resolution: 0.6, nmi: 0.7223324108370404
Cluster for cluster_0.8 with leiden
resolution: 0.8, nmi: 0.7282181046400614
Cluster for cluster_1.0 with leiden
resolution: 1.0, nmi: 0.7318487787801548
Cluster for cluster_1.2 with leiden
resolution: 1.2, nmi: 0.7479404097972805
Cluster for cluster_1.4 with leiden
resolution: 1.4, nmi: 0.7479404097972805
Cluster for cluster_1.6 with leiden
resolution: 1.6, nmi: 0.7541341676577722
Cluster for cluster_1.8 with leiden
resolution: 1.8, nmi: 0.7541341676577722
Cluster for cluster_2.0 with leiden
resolution: 2.0, nmi: 0.7541341676577722
optimised clustering against cell_type
optimal cluster resolution: 1.6
optimal score: 0.7541341676577722
Cluster for cluster_0.2 with leiden
resolution: 0.2, nmi: 0.6573248380137732
Cluster for cluster_0.4 with leiden
resolution: 0.4, nmi: 0.6839811140751546
Cluster for cluster_0.6 with leiden
resolution: 

In [8]:
adata_list

[AnnData object with n_obs × n_vars = 192 × 20898
     obs: 'donor_id', 'Level 1', 'study', 'sex', 'age', 'disease', 'disease_category', 'age_category', 'age_ontology', 'blasts_pct', 'risk', 'psbulk_n_cells', 'psbulk_counts', 'cell_type', 'cluster_0.2', 'cluster_0.4', 'cluster_0.6', 'cluster_0.8', 'cluster_1.0', 'cluster_1.2', 'cluster_1.4', 'cluster_1.6', 'cluster_1.8', 'cluster_2.0', 'cluster'
     var: 'gene_ids', 'gene_count', 'ensembl_gene_id', 'entrezgene_id', 'external_gene_name', 'hgnc_symbol', 'description', 'chromosome_name', 'start_position', 'end_position', 'gene_biotype', 'strand', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
     uns: 'neighbors', 'cluster_0.2', 'cluster_0.4', 'cluster_0.6', 'cluster_0.8', 'cluster_1.0', 'cluster_1.2', 'cluster_1.4', 'cluster_1.6', 'cluster_1.8', 'cluster_2.0'
     obsm: 'pca'
     layers: 'psbulk_props'
     obsp: 'distances', 'connectivities',
 AnnData objec

In [9]:
results = []

for ad in adata_list:
    
    asw= scib.me.silhouette(ad, label_key="cell_type", embed="pca")
    results.append(asw)    

df_ASW = pd.DataFrame(results, columns=['ASW'])

df_ASW['Dataset'] = samples

df_ASW = df_ASW[['Dataset', 'ASW']]

In [10]:
df_ASW

,Dataset,ASW
0,CC24,0.551067
1,hvg24,0.572546
2,receptors24,0.565450
3,random24,0.417569


In [12]:
results = []

for ad in adata_list:
    
    ari = scib.me.ari(ad, cluster_key="cluster", label_key="cell_type")
    results.append(ari)    

df_ARI = pd.DataFrame(results, columns=['ARI'])

df_ARI['Dataset'] = samples

df_ARI = df_ARI[['Dataset', 'ARI']]
df_ARI

,Dataset,ARI
0,CC24,0.364716
1,hvg24,0.438444
2,receptors24,0.462521
3,random24,0.188113


In [13]:
results = []

for ad in adata_list:
    
    nmi = scib.me.nmi(ad, cluster_key="cluster", label_key="cell_type")
    results.append(nmi)    

df_NMI = pd.DataFrame(results, columns=['NMI'])
df_NMI['Dataset'] = samples

df_NMI = df_NMI[['Dataset', 'NMI']]
df_NMI

,Dataset,NMI
0,CC24,0.726361
1,hvg24,0.754134
2,receptors24,0.774564
3,random24,0.507560


In [16]:
merged_df = pd.merge(df_ASW, df_ARI, on="Dataset")
merged_df = pd.merge(merged_df, df_NMI, on="Dataset")
merged_df

,Dataset,ASW,ARI,NMI
0,CC24,0.551067,0.364716,0.726361
1,hvg24,0.572546,0.438444,0.754134
2,receptors24,0.565450,0.462521,0.774564
3,random24,0.417569,0.188113,0.507560


In [22]:
merged_df.to_csv("./data/PCA_perf.csv", index=False)